In [ ]:
from sklearn.svm import SVC
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
import sklearn.metrics as metrics
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import GridSearchCV
from glob import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay

#Se elijen las carpetas desde donde se trabaja
mainpathlocation = 'D:/CosasUniversidad/Material/ULA6-Semestre12/Taller-de-Titulo-II/Recursos/HAM10000/'
imagetrainlocation = 'HAMTESTS/n6Resized'
dataaugmentloc = 'D:/CosasUniversidad/Material/ULA6-Semestre12/Taller-de-Titulo-II/Recursos/HAM10000/HAMTESTS'

In [ ]:
os.chdir(dataaugmentloc)
#Se lee el archivo .pkl obtenido en svmIndexCalc y se declaran los atributos con los que se trabajan
metadf = pd.read_pickle('svmDataset.pkl')
metadf['indexes'] = metadf[['Circularidad', 'ThinRatio', 'AspectRatio','AvgRed','AvgGreen','AvgBlue','AvgHue','AvgSaturation','AvgIntensity','Diameter','Asimetria']].values.tolist()


X = np.asarray(metadf['indexes'].tolist())
Y = metadf['label']
Y_cat = to_categorical(Y, num_classes=7)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.2,
                                                    random_state=12)

#y_train = y_train[0, :] 

#Se entrena usando Linear Discriminant Analysis

lda = LDA()
x_train = lda.fit_transform(x_train, y_train)
x_test = lda.transform(x_test)


In [ ]:
#Se buscan los parametros optimos

parameters = [{'kernel': ['rbf'],
               'gamma': [0.01, 0.1, 0.5],
               'C': [10, 100, 1000]}]
print("# Tuning hyper-parameters")
clf = GridSearchCV(SVC(), parameters, cv=5)
clf.fit(x_train, y_train)
print('best parameters:')
print(clf.best_params_)
print('-------------------------------------')
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"

          % (mean, std * 2, params)) 

In [ ]:
#Se declaran el C y Gamma optimos, obtenidos anteriormente
#noresampling
#optimal_C = 10
#optimal_gamma = 0.1

#SVM 1500
optimal_C = 1000
optimal_gamma = 0.5

#fit model
svc = SVC(kernel="rbf", gamma=optimal_gamma, C=optimal_C)
svc.fit(x_train, y_train)

pred = svc.predict(x_test)
print("La presición del modelo es de: ", accuracy_score(y_test, pred))

In [ ]:
#Se guarda el modelo

import pickle

os.chdir(dataaugmentloc)

# save
with open('svcTrained.pkl','wb') as f:
    pickle.dump(svc,f)

# load
#with open('model.pkl', 'rb') as f:
#    svc2 = pickle.load(f)

#svc2.predict(X[0:1])

In [ ]:
#Opcionalmente, se imprimen graficos mostrando los datos encontrados

ConfusionMatrixDisplay.from_predictions(y_test, pred,labels=[0,1,2,3,4,5,6])
plt.show()

confusion_matrix = metrics.confusion_matrix(y_true=y_test, y_pred=pred )
label_frac_error = 1 - np.diag(confusion_matrix) / np.sum(confusion_matrix, axis=1)
plt.bar(np.arange(7),label_frac_error)
plt.title('Fracción de predicciónes incorrectas')
plt.xlabel('Clase correcta')
plt.ylabel('Fracción clasificada incorrectamente')
plt.show()